In [3]:
import datetime
import dask.dataframe as dd
import dask_geopandas as dg
import dask
import geopandas as gpd
import pandas as pd
from scripts import vp_spatial_accuracy
from segment_speed_utils import helpers
from calitp_data_analysis.geography_utils import WGS84
from segment_speed_utils.project_vars import (
    PROJECT_CRS,
    SEGMENT_GCS,
    analysis_date,
    GCS_FILE_PATH,
    COMPILED_CACHED_VIEWS,
    RT_SCHED_GCS,
    CONFIG_PATH
)

from typing import Literal
import numpy as np

from shared_utils.rt_utils import MPH_PER_MPS
from calitp_data_analysis import utils

# cd rt_segment_speeds && pip install -r requirements.txt && cd
from shared_utils import portfolio_utils, schedule_rt_utils
from segment_speed_utils import helpers, sched_rt_utils, wrangle_shapes, segment_calcs

In [4]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### All operators

#### Links
* https://github.com/cal-itp/data-analyses/blob/main/rt_segment_speeds/scripts/shapely_project_vp.py#L61

#### Ideas
* Calculate averages for night, daytime, peak.

#### Map partitions
* Must specify data type
* Exclude any cols I don't want in meta.
* Other examples of using `persist` and `compute`?

### Spatial Accuracy
* Should this be one big function?

In [5]:
stop

NameError: name 'stop' is not defined

In [ ]:
# 14,514,960 rows
vp_usable= dd.read_parquet(
      f"{SEGMENT_GCS}vp_usable_{analysis_date}"
)

In [ ]:
# calitp-analytics-data/data-analyses/rt_segment_speeds/vp_usable_2023-10-11
operator = "Bay Area 511 Muni VehiclePositions"
gtfs_key = "7cc0cb1871dfd558f11a2885c145d144"

In [ ]:
"""
vp_usable= dd.read_parquet(
    f"{SEGMENT_GCS}vp_usable_{analysis_date}",
    filters=[
        [
            ("gtfs_dataset_name", "==", operator),
            ("schedule_gtfs_dataset_key", "==", gtfs_key),
        ]
    ],
)
"""

In [ ]:
# vp_usable[['gtfs_dataset_name']].compute().nunique()

In [ ]:
def grab_shape_keys_in_vp(vp_usable: dd.DataFrame, analysis_date: str) -> pd.DataFrame:
    """
    Subset raw vp and find unique trip_instance_keys.
    Create crosswalk to link trip_instance_key to shape_array_key.
    """
    vp_usable = vp_usable[['trip_instance_key']]
    
    vp_usable = vp_usable.drop_duplicates().reset_index(drop=True)

    # Make sure we have a shape geometry too
    # otherwise map_partitions will throw error
    shapes = (
        pd.read_parquet(
            f"{COMPILED_CACHED_VIEWS}routelines_{analysis_date}.parquet",
            columns=["shape_array_key"],
        )
        .dropna()
        .drop_duplicates()
    )

    trips_with_shape = (
        helpers.import_scheduled_trips(
            analysis_date,
            columns=["trip_instance_key", "shape_array_key"],
            get_pandas=True,
        )
        .merge(shapes, on="shape_array_key", how="inner")
        .merge(vp_usable, on="trip_instance_key", how="inner")
        .drop_duplicates()
        .dropna()
        .reset_index(drop=True)
    )
    
    trips_with_shape = trips_with_shape[["trip_instance_key", "shape_array_key"]]

    return trips_with_shape

In [ ]:
trips_with_shape = vp_usable.map_partitions(
        grab_shape_keys_in_vp,
        analysis_date,
        meta = {"trip_instance_key": "object",
               "shape_array_key": "object"},
        align_dataframes = False
    ).persist()

In [ ]:

len(trips_with_shape)

#### What do you do if you have two dfs in the argument?

In [ ]:
def buffer_shapes2(
    trips_with_shape_subset: pd.DataFrame,
    analysis_date: str,
    buffer_meters: int = 35,
    **kwargs
) -> gpd.GeoDataFrame:
    """
    Filter scheduled shapes down to the shapes that appear in vp.
    Buffer these.
    
    Attach the shape geometry for a subset of shapes or trips.
    """
    subset = trips_with_shape.shape_array_key.unique()
    subset = subset.compute().tolist()
    
    shapes = helpers.import_scheduled_shapes(
        analysis_date,
        columns = ["shape_array_key", "geometry"],
        filters = [[("shape_array_key", "in", subset)]],
        crs = PROJECT_CRS,
        get_pandas = True
    )
    
    # to_crs takes awhile, so do a filtering on only shapes we need
    shapes = shapes.assign(
        geometry = shapes.geometry.buffer(buffer_meters)
    )
    
    trips_with_shape_geom = pd.merge(
        shapes,
        trips_with_shape_subset,
        on = "shape_array_key",
        how = "inner"
    )
    
    return trips_with_shape_geom


In [ ]:
"""
buffer_shapes = trips_with_shape.map_partitions(
        buffer_shapes2,
        analysis_date,
        35,
        meta = {
               "shape_array_key": "object",
               "geometry":"geometry",
                "trip_instance_key": "object"},
        align_dataframes = False
    ).persist()
    """

In [ ]:
len(buffer_shapes)

In [ ]:
def merge_vp_with_shape_and_count(
    vp: dd.DataFrame, trips_with_shape_geom: gpd.GeoDataFrame
) -> gpd.GeoDataFrame:
    """
    Merge vp with crosswalk and buffered shapes.
    Get vp count totals and vp within shape.
    """
    # COMMENT BACK IN ONCE I MERGE WITH MAIN
    # vp_gdf = wrangle_shapes.vp_as_gdf(vp)

    vp_gdf = gpd.GeoDataFrame(
        vp, geometry=gpd.points_from_xy(vp.x, vp.y), crs=WGS84
    ).to_crs(PROJECT_CRS)
    
    # NEW
    trips_with_shape_geom = trips_with_shape_geom.set_geometry('geometry')
    trips_with_shape_geom = trips_with_shape_geom.set_crs(PROJECT_CRS)
    vp_gdf = vp_gdf.rename(
        columns={
            0: "gtfs_dataset_name",
            1: "schedule_gtfs_dataset_key",
            2: "trip_id",
            3: "trip_instance_key",
            4: "location_timestamp",
            5: "location_timestamp_local",
            6: "x",
            7: "y",
            8: "vp_idx",
            9: "gtfs_dataset_key",
            10: "vp_dir_xnorm",
            11: "vp_dir_ynorm",
            12: "vp_primary_direction",
        }
    )

    vp2 = pd.merge(
        vp_gdf, trips_with_shape_geom, on="trip_instance_key", how="inner"
    ).reset_index(drop=True)

    total_vp = vp_spatial_accuracy.total_vp_counts_by_trip(vp2)

    vp2 = vp2.assign(is_within=vp2.geometry_x.within(vp2.geometry_y)).query(
        "is_within==True"
    )

    vps_in_shape = (
        vp2.groupby("trip_instance_key", observed=True, group_keys=False)
        .agg({"location_timestamp_local": "count"})
        .reset_index()
        .rename(columns={"location_timestamp_local": "vp_in_shape"})
    )

    count_df = pd.merge(total_vp, vps_in_shape, on="trip_instance_key", how="left")

    count_df = count_df.assign(
        vp_in_shape=count_df.vp_in_shape.fillna(0).astype("int32"),
        total_vp=count_df.total_vp.fillna(0).astype("int32"),
    )

    count_df["total_vp_in_shape_pct"] = count_df.vp_in_shape / count_df.total_vp * 100

    return count_df

In [ ]:
# The kernel for data-analyses/rt_scheduled_v_ran/03_metrics_all_ops.ipynb appears to have died. It will restart automatically.
"""
spatial_accuracy_df = vp_usable.map_partitions(
        merge_vp_with_shape_and_count,
        buffer_shapes,
        meta = {'trip_instance_key':'object', 
                'total_vp':'int32', 
                'vp_in_shape':'int32',
                 'total_vp_in_shape_pct':'float64'},
        align_dataframes = False
    ).persist()
    """

In [ ]:
def spatial_accuracy(vp_usable: dd.DataFrame, analysis_date:str):
    trips_with_shape = grab_shape_keys_in_vp(vp_usable, analysis_date)
    
    trips_with_shape_geom = buffer_shapes2(trips_with_shape,analysis_date,  35)

    spatial_accuracy = merge_vp_with_shape_and_count(vp_usable, trips_with_shape_geom)

    return spatial_accuracy

In [ ]:
spatial_accuracy_df = vp_usable.map_partitions(
        spatial_accuracy,
        analysis_date,
        meta = {'trip_instance_key':'object', 
                'total_vp':'int32', 
                'vp_in_shape':'int32',
                 'total_vp_in_shape_pct':'float64'},
        align_dataframes = False
    ).persist()

In [ ]:
len(spatial_accuracy_df)

### Update Completeness

In [ ]:
def pct_of_pings(df: pd.DataFrame):

    # Determine which rows have 2+ pings per minute
    df2 = df.assign(
        atleast2_trip_updates=df.apply(
            lambda x: 1 if x.number_of_pings_per_minute >= 2 else 0, axis=1
        )
    )

    # Calculate total trip time
    df2["max_time"] = df2.location_timestamp_local
    df3 = (
        df2.groupby(["trip_instance_key"])
        .agg(
            {
                "location_timestamp_local": "min",
                "max_time": "max",
                "atleast2_trip_updates": "sum",
            }
        )
        .reset_index()
        .rename(columns={"location_timestamp_local": "min_time"})
    )

    # Add an extra minute
    df3["total_trip_time"] = (df3.max_time - df3.min_time) / pd.Timedelta(minutes=1) + 1

    # Find % of each trip in which one minute has 2+ pings
    df3 = df3.assign(
        pct_with_2_pings_per_min=df3.atleast2_trip_updates.divide(df3.total_trip_time)
        * 100
    )

    return df3

In [ ]:
def density_of_pings(df: pd.DataFrame):

    # Count number of pings that occur
    # on average per 5 minutes of the trip
    df2 = (
        df.groupby(
            [
                *["trip_instance_key"],
                pd.Grouper(key="location_timestamp_local", freq="5Min"),
            ]
        )
        .sum()
        .reset_index()
        .rename(columns={"number_of_pings_per_minute": "number_of_pings_per_5_min"})
    )

    df3 = (
        df2.groupby(["trip_instance_key"])
        .agg({"number_of_pings_per_5_min": "median"})
        .reset_index()
    )

    df3["avg_pings_per_5_min"] = df3.number_of_pings_per_5_min / 5

    return df3

In [ ]:
def total_minutes_with_gtfs(df: pd.DataFrame):
    """
    Total minutes of a trip that recorded any
    vehicle positions.
    """
    df2 = (
        df.groupby(["trip_instance_key"])
        .agg({"gtfs_dataset_key": "count"})
        .rename(columns={"gtfs_dataset_key": "total_minutes_w_gtfs"})
        .reset_index()
    )
    return df2

In [ ]:
def update_completeness(df: pd.DataFrame):
    
    complete_cols = [
    "vp_idx",
    "location_timestamp_local",
    "trip_instance_key",
    "gtfs_dataset_key"]
    
    df = df[complete_cols]
    
    # Find number of pings per minute for each trip
    df = df.sort_values(["vp_idx"]).reset_index(drop=True)

    df2 = (
        df.groupby(
            [
                *["trip_instance_key"],
                pd.Grouper(key="location_timestamp_local", freq="1Min"),
            ]
        )
        .count()
        .reset_index()
        .rename(columns={"vp_idx": "number_of_pings_per_minute"})
    )

    pings_df = pct_of_pings(df2)
    density_df = density_of_pings(df2)
    total_minutes_df = total_minutes_with_gtfs(df2)

    m1 = pings_df.merge(density_df, on="trip_instance_key", how="inner").merge(
        total_minutes_df, on="trip_instance_key", how="inner"
    )

    # m1["pct_of_trip_w_gtfs"] = m1.total_minutes_w_gtfs / m1.total_trip_time * 100
    m1 = m1.drop(columns=["min_time", "max_time", "number_of_pings_per_5_min"])
    return m1


In [ ]:
update_completeness_df = vp_usable.map_partitions(
       update_completeness,
        meta = {'trip_instance_key':'object', 
                'atleast2_trip_updates':'int64', 
                'total_trip_time':'float64',
                 'pct_with_2_pings_per_min':'float64',
               'avg_pings_per_5_min':'float64',
               'total_minutes_w_gtfs':'float64'},
        align_dataframes = False
    ).persist()

In [ ]:
type(update_completeness_df)

In [ ]:
len(update_completeness_df)

In [ ]:
# update_completeness_df2 = update_completeness_df.compute()

In [ ]:
# update_completeness_df2.head()

### Speeds

In [ ]:
trip_speeds = dd.read_parquet(
    f"{SEGMENT_GCS}trip_summary/trip_speeds_{analysis_date}.parquet",
    columns=[
        "trip_instance_key",
        "speed_mph",
        "route_id",
        "time_of_day",
        "service_minutes",
    ],
)

In [ ]:
entire_df = (dd.merge(trip_speeds, spatial_accuracy_df, on="trip_instance_key", how="outer").merge(update_completeness_df,  on = "trip_instance_key", how = "outer"))
                                                                                            

In [ ]:
entire_df.columns

In [ ]:
type(entire_df)

In [ ]:
entire_df = entire_df.compute()

### To do
* Rename pct_with_2_pings_per_min
* Rename atleast2_trip_updates to be something clearer
* Delete total_vp and vp_in_shape


Questions
* Should I bring in gtfs schedule key in `trip_speeds`
* It took a very long time to run
* Rounding guide?
* What to do with rows with `nans` because I did outer joins?
    * 12% of the row don't have speed_mph information
* How to clean it up? 

In [6]:
entire_df = pd.read_parquet('./ah_test')

In [7]:
entire_df = entire_df.drop(columns = ['total_vp','vp_in_shape'])

In [19]:
    entire_df.sample(5)

,trip_instance_key,speed_mph,route_id,time_of_day,service_minutes,total_vp_in_shape_pct,atleast2_trip_updates,total_trip_time,pct_with_2_pings_per_min,avg_pings_per_5_min,total_minutes_w_gtfs,trip_timeliness_pct,gtfs_coverage_v_total_trip_time_pct
210,41f13ebf102673d1779608397ee6a366,10.23,10954,PM Peak,59.00,100.00,103,114.00,90.35,2.40,113,95,99.12
273,576653e3da910885fb413e4507adcf25,11.11,933,Midday,66.00,100.00,44,86.00,51.16,1.60,86,30,100.00
455,8ecc4b434ce0d81c4f3c562e976e4d67,12.36,388,Midday,80.00,87.71,100,114.00,87.72,3.00,102,45,89.47
347,6be6a1f2557133316c295bf73acff6a4,6.59,22,PM Peak,61.00,100.00,64,72.00,88.89,2.00,72,20,100.00
40,0bcce5211e04b8dc7e4cd93fc55501f9,11.46,3635,Midday,74.00,100.00,17,81.00,20.99,1.00,63,10,77.78


In [9]:
entire_df.loc[entire_df.trip_instance_key == "9323cb6c9e3babc5bc8420cd960affee"]

,trip_instance_key,speed_mph,route_id,time_of_day,service_minutes,total_vp_in_shape_pct,atleast2_trip_updates,total_trip_time,pct_with_2_pings_per_min,avg_pings_per_5_min,total_minutes_w_gtfs
859,9323cb6c9e3babc5bc8420cd960affee,NaN,None,None,NaN,NaN,54,55.00,98.18,3.00,55


In [10]:
entire_df.total_vp_in_shape_pct.describe()

count   77091.00
mean       93.98
std        12.79
min         0.00
25%        95.16
50%       100.00
75%       100.00
max       100.00
Name: total_vp_in_shape_pct, dtype: float64

In [11]:
entire_df.pct_with_2_pings_per_min.describe()

count   86486.00
mean       86.34
std        22.40
min         0.00
25%        83.72
50%        97.50
75%       100.00
max       100.00
Name: pct_with_2_pings_per_min, dtype: float64

In [12]:
# entire_df.to_parquet('./ah_test')

### Trip Timeliness
* Using % to measure timeliness
    * A trip that is scheduled to run 30 minutes but has 34 minutes of gtfs data is 15% slower than it's "supposed" to be.
* What to do with extreme values.
    * Service minutes is 12, but trip time is 1,440 minutes.

In [13]:
entire_df["trip_timeliness_pct"] = (entire_df.total_trip_time / entire_df.service_minutes - 1) * 100

In [14]:
entire_df.trip_timeliness_pct = (entire_df.trip_timeliness_pct / 5).fillna(0).round().astype(int) * 5

In [15]:
entire_df.trip_timeliness_pct.describe()

count   86486.00
mean       38.50
std       219.09
min       -85.00
25%         5.00
50%        20.00
75%        40.00
max     11900.00
Name: trip_timeliness_pct, dtype: float64

In [21]:
entire_df[["trip_timeliness_pct", "total_trip_time", "total_minutes_w_gtfs","service_minutes"]].sample(10)

,trip_timeliness_pct,total_trip_time,total_minutes_w_gtfs,service_minutes
824,20,93.00,93,76.00
387,5,35.00,35,33.00
672,20,168.00,162,141.00
524,25,87.00,87,70.00
549,50,50.00,49,33.00
152,60,77.00,77,48.00
395,50,111.00,111,75.00
585,50,102.00,102,68.00
500,25,120.00,115,97.00
284,35,72.00,72,54.00


### Metric: GTFS Coverage
#### Comparing `total_trip_time` with `total_minutes_w_gtfs`
* The trip can run from 11 am to 12 pm but skip recording data in between 11:30-11:45. 
    * Thus the total trip time would be one hour but minutes with gtfs would be 45 minutes.
* On the otherhand, a trip can run for much longer (by accident)
    * As seen  below in which the trip is 11,900% faster...

In [17]:
entire_df['gtfs_coverage_v_total_trip_time_pct'] = (entire_df.total_minutes_w_gtfs/entire_df.total_trip_time) * 100

In [20]:
# Rounds up
entire_df['gtfs_coverage_v_total_trip_time_pct'] = (entire_df.gtfs_coverage_v_total_trip_time_pct / 5).fillna(0).round().astype(int) * 5

In [22]:
entire_df['gtfs_coverage_v_total_trip_time_pct'].describe()

count   86486.00
mean       96.71
std        11.00
min         0.00
25%       100.00
50%       100.00
75%       100.00
max       100.00
Name: gtfs_coverage_v_total_trip_time_pct, dtype: float64

In [23]:
entire_df['gtfs_coverage_v_total_trip_time_pct'].value_counts()

100    71668
95      5730
90      2503
85      1484
80       991
75       767
70       717
65       443
35       366
40       312
60       308
30       238
55       234
45       166
50       165
25       113
5        111
20        60
15        50
10        46
0         14
Name: gtfs_coverage_v_total_trip_time_pct, dtype: int64

In [ ]:
# entire_df[['gtfs_coverage_v_total_trip_time_rounded', 'gtfs_coverage_v_total_trip_time']].sample(10)

##### One trip - extreme
* First time stamp is at 7am on 10/11.
* Last timestamp is 10/12 at 7am.

In [24]:
# 14,514,960 rows
#vp_usable_pd = pd.read_parquet(
#      f"{SEGMENT_GCS}vp_usable_{analysis_date}"
#)

In [ ]:
# vp_usable_pd.loc[vp_usable_pd.trip_instance_key == "a86db56c6c67f9ae4c1a4c828d5e9f91"].sort_values(['location_timestamp'])

In [25]:
entire_df.loc[entire_df.trip_timeliness_pct == 11900]

,trip_instance_key,speed_mph,route_id,time_of_day,service_minutes,total_vp_in_shape_pct,atleast2_trip_updates,total_trip_time,pct_with_2_pings_per_min,avg_pings_per_5_min,total_minutes_w_gtfs,trip_timeliness_pct,gtfs_coverage_v_total_trip_time_pct
531,a86db56c6c67f9ae4c1a4c828d5e9f91,23.17,None,PM Peak,12.00,100.00,10,1440.00,0.69,3.00,11,11900,0
234,4ab96634f0d16bff552b0d50683f83a5,0.05,None,PM Peak,12.00,94.04,50,1440.00,3.47,3.00,51,11900,5


In [ ]:
# entire_df = entire_df.fillna(0)

#### How does the total trip time recorded by GTFS  compare to scheduled service minutes?
* Too similar to trip timeliness?

In [26]:
entire_df['gtfs_coverage_v_scheduled_pct'] = (entire_df.total_minutes_w_gtfs / entire_df.service_minutes * 100)

In [27]:
# Mask any value above 100 because often, total trip time is slower than scheduled
entire_df.gtfs_coverage_v_scheduled_pct = entire_df.gtfs_coverage_v_scheduled_pct.mask(entire_df.gtfs_coverage_v_scheduled_pct > 100).fillna(100)

In [28]:
entire_df.gtfs_coverage_v_scheduled_pct = (entire_df.gtfs_coverage_v_scheduled_pct / 5).fillna(0).round().astype(int) * 5

In [29]:
entire_df.gtfs_coverage_v_scheduled_pct.describe()

count   86486.00
mean       97.70
std         9.11
min         0.00
25%       100.00
50%       100.00
75%       100.00
max       100.00
Name: gtfs_coverage_v_scheduled_pct, dtype: float64

In [30]:
entire_df.gtfs_coverage_v_scheduled_pct.value_counts()

100    76744
95      2956
90      1885
85      1158
80       792
75       598
70       392
45       298
40       280
65       269
50       230
55       209
60       208
35       172
30       113
25        73
20        38
15        28
10        25
5         14
0          4
Name: gtfs_coverage_v_scheduled_pct, dtype: int64

In [31]:
entire_df.loc[entire_df.trip_timeliness_pct == 11900]

,trip_instance_key,speed_mph,route_id,time_of_day,service_minutes,total_vp_in_shape_pct,atleast2_trip_updates,total_trip_time,pct_with_2_pings_per_min,avg_pings_per_5_min,total_minutes_w_gtfs,trip_timeliness_pct,gtfs_coverage_v_total_trip_time_pct,gtfs_coverage_v_scheduled_pct
531,a86db56c6c67f9ae4c1a4c828d5e9f91,23.17,None,PM Peak,12.00,100.00,10,1440.00,0.69,3.00,11,11900,0,90
234,4ab96634f0d16bff552b0d50683f83a5,0.05,None,PM Peak,12.00,94.04,50,1440.00,3.47,3.00,51,11900,5,100


* Some trips are apparently more than 35% faster than their scheduled service minutes, which seems not very likely? 
* The example trips below
    * Faster than service minutes by 45% and 75%.
    * Every minute has more than 2 GTFS ping, so the data quality looks good

In [32]:
# Filter out
faster_than_scheduled = entire_df.loc[entire_df.trip_timeliness_pct < 0]

In [33]:
faster_than_scheduled.trip_timeliness_pct.describe()

count   6441.00
mean     -14.35
std       13.52
min      -85.00
25%      -20.00
50%      -10.00
75%       -5.00
max       -5.00
Name: trip_timeliness_pct, dtype: float64

In [34]:
faster_than_scheduled.loc[faster_than_scheduled.trip_instance_key == "d97697de370980ad54f378ac1ee49142"]

,trip_instance_key,speed_mph,route_id,time_of_day,service_minutes,total_vp_in_shape_pct,atleast2_trip_updates,total_trip_time,pct_with_2_pings_per_min,avg_pings_per_5_min,total_minutes_w_gtfs,trip_timeliness_pct,gtfs_coverage_v_total_trip_time_pct,gtfs_coverage_v_scheduled_pct
705,d97697de370980ad54f378ac1ee49142,18.42,85,Midday,46.00,100.00,24,25.00,96.00,3.00,25,-45,100,55


In [35]:
faster_than_scheduled.loc[faster_than_scheduled.trip_instance_key == "2d639eaffc7c93b9e020a1e7bb93edfa"]

,trip_instance_key,speed_mph,route_id,time_of_day,service_minutes,total_vp_in_shape_pct,atleast2_trip_updates,total_trip_time,pct_with_2_pings_per_min,avg_pings_per_5_min,total_minutes_w_gtfs,trip_timeliness_pct,gtfs_coverage_v_total_trip_time_pct,gtfs_coverage_v_scheduled_pct
156,2d639eaffc7c93b9e020a1e7bb93edfa,26.18,603,PM Peak,50.00,100.00,13,13.00,100.00,2.80,13,-75,100,25


In [36]:
faster_than_scheduled.loc[faster_than_scheduled.trip_timeliness_pct == -35].sample(3)

,trip_instance_key,speed_mph,route_id,time_of_day,service_minutes,total_vp_in_shape_pct,atleast2_trip_updates,total_trip_time,pct_with_2_pings_per_min,avg_pings_per_5_min,total_minutes_w_gtfs,trip_timeliness_pct,gtfs_coverage_v_total_trip_time_pct,gtfs_coverage_v_scheduled_pct
302,4c169f202c7da738a95212cc2f00b5dc,35.00,JPX,AM Peak,26.00,100.00,16,17.00,94.12,2.90,17,-35,100,65
289,56dac7df866fa5f50e36fbdfe17f41a6,8.07,LBUS,PM Peak,18.00,50.00,6,12.00,50.00,0.80,9,-35,75,50
43,0cb77daa7eb61919d2c734b985587e1d,6.95,86,PM Peak,25.00,100.00,9,16.00,56.25,1.40,15,-35,95,60


In [37]:
faster_than_scheduled.trip_timeliness_pct.value_counts()

-5     2617
-10    1449
-15     755
-20     468
-25     331
-30     216
-35     160
-40     111
-45      69
-50      58
-55      52
-60      48
-65      39
-70      27
-75      21
-80      16
-85       4
Name: trip_timeliness_pct, dtype: int64